Importing Libraries

In [2]:
from pydriller import Repository
import csv
import pandas as pd
import torch
import numpy as np
import matplotlib.pyplot as plt

Importing Repo URL

In [33]:
repo_url = 'https://github.com/pythonprofilers/memory_profiler.git'

repo = Repository(repo_url)

Mining Bug fixing commits

In [14]:
keywords = ['fix', 'fixes', 'fixed', 'bug', 'bugfix', 'defect', 'defects',
            'error', 'issue', 'patch', 'repair', 'fault', 'resolved', 'resolve']

def is_a_bug_fixing_commit(msg):
    if msg is None or msg == "":
        return False
    msg = msg.lower()
    for keyword in keywords:
        if keyword in msg:
            return True
    return False

with open("bug_fixing_commits.csv", mode='w', newline='', encoding='utf-8') as file:
    write = csv.writer(file)
    write.writerow(["Hash", "Message", "Hashes of Parents",
                    "Is a merge commit", "List of modified files"])
    
    for commit in repo.traverse_commits():
        msg = commit.msg.lower()
        if is_a_bug_fixing_commit(msg) == True:

            parents = commit.parents

            if len(commit.parents) > 1:
                is_merge = "Yes"
            else:
                is_merge = "No"

            modif_file = []
            for mod in commit.modified_files:
                if mod.new_path:
                    modif_file.append(mod.new_path)

            write.writerow([commit.hash, commit.msg.strip().replace("\n"," "),
                           parents, is_merge, modif_file])

pd.set_option("display.width", 200) 
pd.set_option("display.max_colwidth", 20)
df = pd.read_csv("bug_fixing_commits.csv")
df = pd.DataFrame(df)
print(df)

                    Hash              Message    Hashes of Parents Is a merge commit List of modified files
0    8800fe9252802753...  Port to OSX and ...  ['76745998505624...                No  ['examples/plot_...  
1    c953ca4a9df3a332...  FIX: display las...  ['8af133fea44642...                No  ['memory_profile...  
2    ec9ee7032b01cf93...  Fix typos in REA...  ['992bb0a8c911e1...                No  ['README.rst', '...  
3    4214cfc13d033c43...  FIX: README.rst ...  ['ea5d79bd10b52b...                No  ['MANIFEST', 'me...  
4    303cab8df4b2274c...  Explicit error f...  ['7a9ec119e19086...                No  ['memory_profile...  
..                   ...                  ...                  ...               ...                  ...  
196  dd9ed705449638f4...  Merge pull reque...  ['4b81aee7064ba4...               Yes                   []  
197  940aa76f5da8b7cb...  Fix deprecated s...  ['57b643a84e1b7f...                No  ['test/test_exit...  
198  e079d3fa35188908...  Fi

Part e

In [16]:
with open("diff_extraction.csv", mode='w',newline='', encoding='utf-8') as file2:
    write = csv.writer(file2)
    write.writerow([
        "Hash", "Message", "Filename",
        "Source Code (before)", "Source Code (current)",
        "Diff", "LLM Inference (fix type)", "Rectified Message"
    ])
    
    for commit in repo.traverse_commits():
        if is_a_bug_fixing_commit(commit.msg):
            for mod in commit.modified_files:
                write.writerow([ commit.hash, commit.msg.strip().replace("\n", " "), mod.new_path, 
                                mod.source_code_before if mod.source_code_before else "",
                                mod.source_code if mod.source_code else "", mod.diff if mod.diff else "", "", ""])

pd.set_option("display.width", 320)
pd.set_option("display.max_colwidth", 40)
df = pd.read_csv("diff_extraction.csv")
df = pd.DataFrame(df)
print(df)


                                        Hash                                  Message                           Filename                     Source Code (before)                    Source Code (current)                                     Diff  LLM Inference (fix type)  Rectified Message
0    8800fe9252802753643a0e48de52c9d8d12f...              Port to OSX and other fixes            examples/plot_memory.py  """\nPlot memory usage of a numeric ...  """\nPlot memory usage of a numeric ...  @@ -1,38 +1,18 @@\n """\n Plot memor...                       NaN                NaN
1    8800fe9252802753643a0e48de52c9d8d12f...              Port to OSX and other fixes                 memory_profiler.py  """Get process information"""\n\n__v...  """Get process information"""\n\n__v...  @@ -6,17 +6,18 @@ _CMD_USAGE = "pyth...                       NaN                NaN
2    c953ca4a9df3a3321166803752608b70e2f2...              FIX: display last statement                 memory_profiler.py  """Get proc

Loading the LLM

In [3]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("mamiksik/CommitPredictorT5")
model = AutoModelForSeq2SeqLM.from_pretrained("mamiksik/CommitPredictorT5")

/Users/arinmehta/Documents/GitHub/STT/Assignment-1/pydriller_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using the LLM on the Diff

In [5]:
df = pd.read_csv("diff_extraction.csv")

for i,row in df.iterrows():
    diff_text = str(row['Diff']).strip()

    cur = str(row['LLM Inference (fix type)']).strip()
    df.at[i, 'Rectified Message'] = cur

    if cur != "" and cur.lower() != "nan":
        continue

    if diff_text.strip()=="":
        df.at[i, 'LLM Inference (fix type)'] = " "
        df.at[i, 'Rectified Message'] = " "

    else:
        inputs = tokenizer(diff_text, return_tensors="pt", truncation=True, max_length=512)
        outputs = model.generate(**inputs)
        prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
        df.at[i, 'LLM Inference (fix type)'] = prediction 
        df.at[i, 'Rectified Message'] = prediction

df.to_csv("diff_extraction.csv", index=False)

pd.set_option("display.width", 350)
pd.set_option("display.max_colwidth", 40)
df = pd.read_csv("diff_extraction.csv")
df = pd.DataFrame(df)
print(df)


                                        Hash                                  Message                           Filename                     Source Code (before)                    Source Code (current)                                     Diff                LLM Inference (fix type)                       Rectified Message
0    8800fe9252802753643a0e48de52c9d8d12f...              Port to OSX and other fixes            examples/plot_memory.py  """\nPlot memory usage of a numeric ...  """\nPlot memory usage of a numeric ...  @@ -1,38 +1,18 @@\n """\n Plot memor...                   add memory usage plot                   add memory usage plot
1    8800fe9252802753643a0e48de52c9d8d12f...              Port to OSX and other fixes                 memory_profiler.py  """Get process information"""\n\n__v...  """Get process information"""\n\n__v...  @@ -6,17 +6,18 @@ _CMD_USAGE = "pyth...               add docs for memory_usage               add docs for memory_usage
2    c953ca4a9df3a3321166803

In [11]:
trivial = {"fix", "update", "typo", "bug", "minor change", "change"}

def is_precise(msg):
    msg = str(msg).strip().lower()
    if len(msg) < 10: 
        return False
    if msg in trivial:
        return False
    if len(msg) > 200: 
        return False
    return True

df['Dev_Precise'] = df['Message'].apply(is_precise)
df['LLM_Precise'] = df['LLM Inference (fix type)'].apply(is_precise)
df['Rectified_Precise'] = df['Rectified Message'].apply(is_precise)

total_commits = len(df)
dev_hit_rate = df['Dev_Precise'].sum() / total_commits
llm_hit_rate = df['LLM_Precise'].sum() / total_commits
rectified_hit_rate = df['Rectified_Precise'].sum() / total_commits

print("Hit Rates (stricter evaluation):")
print(f"RQ1 (Developer): {dev_hit_rate:.2%}")
print(f"RQ2 (LLM): {llm_hit_rate:.2%}")
print(f"RQ3 (Rectifier): {rectified_hit_rate:.2%}")


Hit Rates (stricter evaluation):
RQ1 (Developer): 89.53%
RQ2 (LLM): 99.42%
RQ3 (Rectifier): 99.42%


In [12]:
hit_table = pd.DataFrame({
    'RQ': ['RQ1 (Developer)', 'RQ2 (LLM)', 'RQ3 (Rectifier)'],
    'Hit Rate (%)': [dev_hit_rate*100, llm_hit_rate*100, rectified_hit_rate*100]
})

print(hit_table)

                RQ  Hit Rate (%)
0  RQ1 (Developer)     89.534884
1        RQ2 (LLM)     99.418605
2  RQ3 (Rectifier)     99.418605
